In [19]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

import time
import json

CHROME_DRIVER_PATH = r'C:/chromedriver/chromedriver.exe'
PAGE_URL = "http://www.law.go.kr/precSc.do?tabMenuId=tab67"
SAVE_PATH = './data/'

driver = webdriver.Chrome(executable_path=CHROME_DRIVER_PATH)
driver.implicitly_wait(5)

In [20]:
def open_main():
    driver.get(PAGE_URL)

def main_to_page():
    container = driver.find_element_by_id("viewHeightDiv")
    tl = container.find_element_by_class_name('tl')
    tl.find_element_by_tag_name('a').click()

def get_corpus_from_page():
    corpus_set = []
    corpus = []
    
    content = driver.find_element_by_class_name("pgroup")
    p = content.find_elements_by_class_name("pty4_dep1")
    h5 = content.find_elements_by_tag_name("h5")
    
    for i in range(len(p)):
        corpus.append(h5[i].text)
        corpus.append(p[i].text)
        corpus_set.append(corpus.copy())
        corpus.clear()
    
    return corpus_set

def get_page_list():
    all_page = driver.find_element_by_id("leftContent")
    page_list = all_page.find_elements_by_tag_name("li")
    
    return page_list

def get_json_form(raw):
    contents = []
    
    for con in raw:
        contents.append({con[0] : con[1]})
    
    return contents

In [21]:
open_main()
main_to_page()

In [ ]:
order_list = []
reason_list = []
json_list = []
visited_page = ['0']
first_page = True
last_page = False
terminal = False
corrupted_count = 0
corrupted_flag = False

while True:
    flag = True
    while flag:
        try:
            all_pages = get_page_list()
            flag = False
            
        except:
            flag = True   
            
    if all_pages[-1].text == visited_page[-1]:
        last_page = True 
            
    for page in all_pages[:]:
        try:
            if len(page.text) <= 4:
                try:
                    if int(page.text) % 100 == 0:
                        with open(SAVE_PATH + page.text + ".json", 'w') as f:
                            json.dump(json_list, f, ensure_ascii=False)
                        
                except:
                    print("File I/O Error")
                
                if first_page:
                    if page.text == all_pages[-1].text:
                        terminal = True
                    
                    visited_page.append(page.text)
                    first_page = False
                
                if page.text not in visited_page:
                    visited_page.append(page.text)
                    page.click()
                    time.sleep(2)
                    break

            else:
                page.click()

                flag = True
                while flag:
                    try:
                        json_list.append(get_json_form(get_corpus_from_page()))
                        flag = False
                        
                    except:
                        flag = True
                        
        except:
            print("Exception")
            continue
    
    if last_page:
        flag = True
        while flag:
            try:
                paging = driver.find_element_by_class_name("paging")
                a = paging.find_elements_by_tag_name("a")
                flag = False
                
            except:
                flag = True
                
        if len(a) < 6:
            terminal = True
        
        if len(visited_page) <= 6:
            a[4].click()
            
        else:
            a[6].click()
            
        time.sleep(2)
        last_page = False
        first_page = True
        
    if terminal:
        break
            

In [5]:
with open(SAVE_PATH + "judicial-data.json", 'w') as f:
    json.dump(json_list, f, ensure_ascii=False)

In [3]:
with open(SAVE_PATH + "judicial-data.json", 'r') as f:
    json_data = json.load(f)
    
len(json_data)

80216

In [18]:
json_data[29]

[{'【원고, 상고인】': '원고 1 외 3인'},
 {'【피고, 피상고인】': '국민건강보험공단 인천부평지사장 (소송대리인 변호사 장덕규)'},
 {'【원심판결】': '서울고법 2016. 5. 20. 선고 2015누52663 판결'},
 {'【주 문】': '상고를 모두 기각한다. 상고비용은 원고들이 부담한다.'},
 {'【이 유】': '상고이유를 판단한다. \n1.  항고소송의 대상이 되는 행정처분이라 함은 행정청의 공법상 행위로서 특정사항에 대하여 법규에 의한 권리의 설정 또는 의무의 부담을 명하며 기타 법률상 효과를 발생하게 하는 등 국민의 구체적 권리의무에 직접적 변동을 초래하는 행위를 말하고, 행정청 내부에서의 행위나 알선, 권유, 사실상의 통지 등과 같이 상대방 또는 기타 관계자들의 법률상 지위에 직접적인 법률적 변동을 일으키지 아니하는 행위는 항고소송의 대상이 될 수 없다(대법원 2008. 4. 24. 선고 2008두3500 판결 등 참조).\n한편 국민건강보험법 제5조는 국내에 거주하는 국민은 원칙적으로 건강보험의 가입자 또는 피부양자가 된다고 규정하고 있고, 제6조는 건강보험의 가입자를 직장가입자와 지역가입자로 분류하고 있다. 같은 법 제8조 내지 제10조는 직장가입자 또는 지역가입자 자격의 취득·변동 및 상실 시기를 구체적으로 명시하고 있고, 특히 제9조 제1항은 지역가입자는 적용대상사업장의 사용자로 되거나 근로자 등으로 사용된 날에(제1호), 직장가입자는 그 사용관계가 끝난 날의 다음 날에(제3호) 각각 그 자격이 변동된다고 규정하고 있다. 또한 국민건강보험법 제11조는 가입자의 자격의 취득·변동 및 상실은 가입자 자격의 취득·변동 및 상실 시기에 소급하여 효력이 발생하는 것으로 규정하고 있는데(제1항 전문), 다만 가입자 자격의 취득·변동 및 상실 여부 및 시기를 명확하게 하기 위하여 보험자가 그 사실을 확인할 수 있도록 하고(제1항 후문), 가입자 또는 가입자이었던 자도 보험자에게 그 확인을 청구할 수 있는 권리를 부여하고 있다(제2항)